In [1]:
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models, io, utils
from cellpose.io import imread
import imageio.v2 as imageio
from imageio.v2 import imread
import skimage as sk
import napari
import pandas as pd
import os

In [ ]:
red_path = 'Path/to/red/channel'
green_path = 'Path/to/green/channel'
red_images = os.listdir(red_path)
green_images = os.listdir(green_path)
save_df_path = 'Path/to/save/dataframes'
save_imgs_path = 'Path/to/save/image_stacks'
cp_masks_path = 'Path/to/save/CP_masks'
print(red_images)
print(green_images)
#create avg projections of each channel
avg_img_red = []
avg_img_green = []
for i in range(len(red_images)):
    img_red = imread(os.path.join(red_path, red_images[i]))
    np_img_red = np.asarray(img_red, dtype=np.float32)
    avg_red = np.average(np_img_red, axis=0)
    avg_img_red.append(avg_red)
    img_green = imread(os.path.join(green_path, green_images[i]))
    np_img_green = np.asarray(img_green, dtype=np.float32)
    avg_green = np.average(np_img_green, axis=0)
    avg_img_green.append(avg_green)
print('Done creating arrays')
#Create a ratio image of red/green
ratio_imgs = []
for i in range(len(avg_img_green)):
    ratio = np.divide(avg_img_red[i], avg_img_green[i], out=np.zeros_like(avg_img_red[i]), where=avg_img_green[i]!=0)
    ratio_imgs.append(ratio)
print('Done generating ratio image')
#Send ratio images to CP to generate masks
io.logger_setup()
model = models.Cellpose(gpu=True,model_type='cyto3') # model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'
channels =[[0,0]]
masks, flows, styles, diams = model.eval(ratio_imgs, diameter=270, channels=channels)
for i in range(len(masks)):
    sk.io.imsave(os.path.join(cp_masks_path,'ratio_mask_'+red_images[i]),masks[i])
print('Done creating CP masks')
#Stack images and then get measurements
img_stacks = []
for i in range(len(avg_img_green)):
    image_list = [avg_img_green[i],avg_img_red[i],ratio_imgs[i]]
    all_channels = np.stack(image_list, axis=-1)
    img_stacks.append(all_channels)
    print('image ', str(i), ' shape:', all_channels.shape)
    sk.io.imsave(os.path.join(save_imgs_path,'ratio_'+red_images[i]),all_channels)
    cell_masks = sk.measure.label(masks[i]) #does not create a label image in Napari
    ratio_measurements = sk.measure.regionprops_table(cell_masks, img_stacks[i], 
                                                  properties=('label','area', 'centroid','intensity_max','intensity_mean','intensity_min'))
    df = pd.DataFrame.from_dict(ratio_measurements)
    clean_name = str(red_images[i])
    clean_name = clean_name[3:-4]
    df.to_csv(os.path.join(save_df_path,'measurements_'+clean_name+'.csv'))
print('Done with creating measurements!')